**Getting the Data from Site**

In [1]:
import urllib3

# Create a PoolManager instance
http = urllib3.PoolManager()

# Specify the URL you want to request
url = "https://www.gutenberg.org/cache/epub/11/pg11-images.html" #link to the Book
url_2 = "https://www.gutenberg.org/cache/epub/11/pg11.txt" #Link of plain text of the above mentioned book

# Send an HTTP GET request
Data = http.request("GET", url_2)

# Read and print the response content
Data = Data.data.decode('utf-8')


In [2]:
import numpy as np

# load ascii text and covert to lowercase
raw_text = Data
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  167711
Total Vocab:  66


# Prepare the dataset of input to output pairs encoded as integers

In [4]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  167611


# prepare the dataset of input to output pairs encoded as integers

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  167611


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([167611, 100, 1]) torch.Size([167611])


C:\Users\Abhay\AppData\Local\Temp\ipykernel_14472\1591616650.py:6: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
